In [ ]:
import math
import os.path as path
import numpy as np
import pandas as pd
from skimage.util.montage import montage2d
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from keras.utils.np_utils import to_categorical
from keras.models import Sequential, Model
from keras.callbacks import EarlyStopping
from keras.layers import Conv2D, BatchNormalization, Dropout, MaxPooling2D, Dense, Flatten, ZeroPadding2D, Activation
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2
from keras.layers import average, Input, Concatenate
from extra_functions import *
from scipy import signal


def load_and_format(in_path):
    out_df = pd.read_json(in_path)
    out_images = out_df.apply(lambda c_row: [np.stack([c_row['band_1'],c_row['band_2']], -1).reshape((75,75,2))],1)
    out_images = np.stack(out_images).squeeze()
    return out_df, out_images

dir_path = path.abspath(path.join('__file__',"../.."))
train_path = dir_path + "/train.json"
test_path = dir_path + "/test.json"

train_df, train_images = load_and_format(train_path)
test_df, test_images = load_and_format(test_path)
x_angle_test = np.array(test_df.inc_angle)   
    
"""
x_train, x_val, x_angle_train, x_angle_val, y_train, y_val = train_test_split(train_images
                    , x_angle_train, to_categorical(train_df['is_iceberg']), random_state=2017, train_size=0.75)

print('Train', x_train.shape, y_train.shape)
print('Validation', x_val.shape, y_val.shape) """


print("reading data")
x_train = pd.read_csv('x_train_new.csv', header=None)
x_train = x_train.values
y_train = pd.read_csv('y_train_new.csv', header=None)
y_train = y_train.values
x_val = pd.read_csv('x_val_new.csv', header=None)
x_val = x_val.values
y_val = pd.read_csv('y_val_new.csv', header=None)
y_val = y_val.values  
x_angle_train = pd.read_csv('x_angle_train.csv', header=None)
x_angle_train = x_angle_train.values
x_angle_val = pd.read_csv('x_angle_val.csv', header=None)
x_angle_val = x_angle_val.values

print("reshaping data")
x_train = np.reshape(x_train, (2203, 75, 75, 2))
x_val = np.reshape(x_val, (401, 75, 75, 2))

weight_decay = 0.005

image_input = Input(shape=(75, 75, 2), name="image")
angle_input = Input(shape=[1], name="angle")

cnn = BatchNormalization(momentum=0.99)(image_input)

cnn = Conv2D(64, kernel_size=(3,3), padding = 'same', kernel_regularizer=l2(weight_decay))(cnn)
cnn = Activation('relu')(cnn)
cnn = BatchNormalization(momentum=0.99)(cnn)
cnn = MaxPooling2D(pool_size=(2,2))(cnn)
cnn = Dropout(0.3, seed=25)(cnn)

cnn = Conv2D(64, kernel_size=(3,3), padding = 'same', kernel_regularizer=l2(weight_decay))(cnn)
cnn = Activation('relu')(cnn)
cnn = BatchNormalization(momentum=0.99)(cnn)
cnn = MaxPooling2D(pool_size=(2,2))(cnn)
cnn = Dropout(0.3, seed=25)(cnn)

cnn = Conv2D(128, kernel_size=(3,3), padding = 'same', kernel_regularizer=l2(weight_decay))(cnn)
cnn = Activation('relu')(cnn)
cnn = BatchNormalization(momentum=0.99)(cnn)
cnn = MaxPooling2D(pool_size=(2,2))(cnn)
cnn = Dropout(0.3, seed=25)(cnn)

cnn = Conv2D(128, kernel_size=(3,3), padding = 'same', kernel_regularizer=l2(weight_decay))(cnn)
cnn = Activation('relu')(cnn)
cnn = BatchNormalization(momentum=0.99)(cnn)
cnn = MaxPooling2D(pool_size=(2,2))(cnn)
cnn = Dropout(0.3, seed=25)(cnn)

cnn = Conv2D(256, kernel_size=(3,3), padding = 'same', kernel_regularizer=l2(weight_decay))(cnn)
cnn = Activation('relu')(cnn)
cnn = BatchNormalization(momentum=0.99)(cnn)
cnn = MaxPooling2D(pool_size=(2,2))(cnn)
cnn = Dropout(0.5, seed=25)(cnn)

cnn = Conv2D(512, kernel_size=(3,3), padding = 'same', kernel_regularizer=l2(weight_decay))(cnn)
cnn = Activation('relu')(cnn)
cnn = BatchNormalization(momentum=0.99)(cnn)
cnn = MaxPooling2D(pool_size=(2,2))(cnn)
cnn = Dropout(0.5, seed=25)(cnn)

cnn = Flatten()(cnn)
concatenated_features = Concatenate()([cnn, BatchNormalization()(angle_input)])

cnn = Dense(512, activation='relu', kernel_regularizer=l2(weight_decay))(concatenated_features)
cnn = Dropout(0.4, seed=25)(cnn)

cnn = Dense(256, activation='relu', kernel_regularizer=l2(weight_decay))(cnn)
cnn = Dropout(0.4, seed=25)(cnn)

cnn = Dense(75, activation='relu', kernel_regularizer=l2(weight_decay))(cnn)
cnn = Dropout(0.5, seed=25)(cnn)

output = Dense(2, activation='softmax')(cnn)


model = Model(inputs=[image_input, angle_input], outputs=output)
model.compile(optimizer='adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
model.summary()
print("Training")
early_stopping = EarlyStopping(monitor = 'val_loss', patience = 25)
model.fit([x_train, x_angle_train], y_train, batch_size = 64, validation_data = ([x_val, x_angle_val], y_val), 
          epochs = 250, shuffle = True, callbacks=[early_stopping])


print("predicting")
test_predictions = model.predict([test_images, x_angle_test])

pred_df = test_df[['id']].copy()
pred_df['is_iceberg'] = test_predictions[:,1]
print("creating csv")
pred_df.to_csv('predictions_3.csv', index = False)

Using TensorFlow backend.


reading data
reshaping data
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
image (InputLayer)               (None, 75, 75, 2)     0                                            
____________________________________________________________________________________________________
batch_normalization_1 (BatchNorm (None, 75, 75, 2)     8           image[0][0]                      
____________________________________________________________________________________________________
conv2d_1 (Conv2D)                (None, 75, 75, 64)    1216        batch_normalization_1[0][0]      
____________________________________________________________________________________________________
activation_1 (Activation)        (None, 75, 75, 64)    0           conv2d_1[0][0]                   
_______________________________________________________________

Train on 2203 samples, validate on 401 samples
Epoch 1/250
2203/2203 [==============================] - 6s - loss: 7.4122 - acc: 0.5742 - val_loss: 7.3847 - val_acc: 0.5062
Epoch 2/250
2203/2203 [==============================] - 2s - loss: 6.8235 - acc: 0.6060 - val_loss: 6.6727 - val_acc: 0.4938
Epoch 3/250
2203/2203 [==============================] - 2s - loss: 6.3530 - acc: 0.6518 - val_loss: 6.2503 - val_acc: 0.4938
Epoch 4/250
2203/2203 [==============================] - 2s - loss: 5.8834 - acc: 0.6664 - val_loss: 5.7836 - val_acc: 0.4938
Epoch 5/250
2203/2203 [==============================] - 2s - loss: 5.4267 - acc: 0.6754 - val_loss: 5.3362 - val_acc: 0.5387
Epoch 6/250
2203/2203 [==============================] - 2s - loss: 4.9660 - acc: 0.6941 - val_loss: 4.9153 - val_acc: 0.4938
Epoch 7/250
2203/2203 [==============================] - 2s - loss: 4.4955 - acc: 0.7562 - val_loss: 4.5208 - val_acc: 0.5087
Epoch 8/250
2203/2203 [==============================] - 2s - loss: 4.0